Laboratorio 4 - Modelo de Regresión Logística Polinomial

By Cristian Laynez - 201281

El objetivo de este laboratorio es crear un modelo de regresión logística polinomial que prediga fielmente si un paciente sufrirá de un paro cardíaco.


Información Extra: 
para la data set proveida donde se muestran las condiciones físicas y contextuales para más de 4000 pacientes de enfermedades cardíacas. El dataset relaciona a cada paciente con una etiqueta (1 = tuvo un paro cardíaco, 0 = no tuvo paro cardíaco)

Task 1.1 - Leer archivo proporcionado y almacenarlo en un np.array

In [29]:
# Importar todas las librerias a utilizar
import numpy as np
import matplotlib.pyplot as plt
import csv
from numpy.linalg import inv

In [30]:
# --> Leer la data proporcionada
f = open('framingham.csv', 'r')
reader = csv.reader(f)
header = next(reader, None)

# Se guardara la data en un diccionario
data = {}
for h in header:
    data[h] = []

# Se guardaran todos los valores
for row in reader:
    for h, v in zip(header, row):      
        d = 0.0 if v == "NA" else float(v)
        data[h].append(d)
        # temp = [d]
        # data[h].append(temp)

In [31]:
# Imprimir todos los headers que se encontraron
print(header)

['sex', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD']


In [32]:
# Modelos con solo 1's y 0's
print(data['sex']) # hombre (1) o mujer (0)
print(data['currentSmoker']) # Si es fumador o no
print(data['BPMeds']) # Si el paciente esta tomando medicamentos para la presión alta
print(data['prevalentStroke']) # Si el paciente a tenido un derrame cerebral
print(data['prevalentHyp']) # Si el paciente era hipertenso
print(data['diabetes']) # Si el paciente tiene diabetes
print(data['TenYearCHD'])

[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0,

Task 1.2 - Ajustar modelo logístico polinomial (Tomando en cuenta la variable dependiente de salida [sufre o no subre un paro cardíaco])

In [44]:
# --> TODO: Funciones que nos ayudaran a ajustar el modelo logistico polinomial

# Signo sigmoid
sigmoid = lambda z : 1 / (1 + np.exp(-z))

# Predecir probabilidades
predict_prob = lambda X, theta : sigmoid(np.dot(X, theta))

# Predecir las etiquetas impuestas
predict_labels = lambda X, theta, threshold=0.5 : (predict_prob(X, theta) >= threshold).astype(int)

# Costo y gradiente
def cost_and_grad_function(X, y, theta, lambda_reg):
    m = X.shape[0]
    h = sigmoid(np.dot(X, theta))
    cost = -1/m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h))) + lambda_reg/(2*m) * np.sum(np.square(theta[1:]))
    gradient = 1/m * np.dot(X.T, (h - y)) + lambda_reg/m * np.vstack(([[0]], theta[1:]))
    return cost, gradient

# Obtener descenso de gradiente
def gradient_descent(X, y, theta, alpha, num_iters, lambda_reg):
    decline_res = []
    for i in range(num_iters):
        cost, gradient = cost_and_grad_function(X, y, theta, lambda_reg)
        decline_res.append(cost)
        theta -= alpha * gradient
    return theta, decline_res

In [ ]:
def polinomial_logistic(X, degree):
    return

In [42]:
# ---> Preparacion de datoss
# Para X: age, sysBP, cigsPerDay, glucose, prevalentStroke (Ya que son los datos más influyentes para que ocurra un paro cardiaco)
x = np.array([data['age'], data['sysBP'], data['cigsPerDay'], data['glucose'], data['prevalentStroke']]).T

# Para Y : TenYearCHD
y = np.array(data['TenYearCHD'])

In [43]:
# ---> Vamos a dividir los datos en conjuntos de train and test
m = x.shape[0]
m_train = int(m * 0.7)
m_test = m - m_train
X_train = x[:m_train, :]
X_test = x[m_train:, :]
y_train = y[:m_train]
y_test = y[m_train:]

# ----> Se normalizaran los datos para un mejor manejo de datos
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train_norm = (X_train - mean) / std
X_test_norm = (X_test - mean) / std

Task 1.3 - Utilizar implementación vectorial del algoritmo de regresioń logística

Task 1.4 - Determinar el grado del polinomio que mejor describe la nube de puntos (cross-validation)

Task 1.5 - Análisis sobre sus hallazgos